# 01. 프롬프트(Prompt)

## PromptTemplate

In [1]:
from dotenv import load_dotenv

load_dotenv()

True

In [3]:
# LangSmith 추적을 설정합니다. https://smith.langchain.com
# !pip install -qU langchain-teddynote
from langchain_teddynote import logging

# 프로젝트 이름을 입력합니다.
logging.langsmith("CH02-Prompt-Junu")

LangSmith 추적을 시작합니다.
[프로젝트명]
CH02-Prompt-Junu


LLM 객체를 정의합니다.

In [4]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI()

## 방법 1. from_template() 메소드를 사용하여 PromptTemplate 객체 생성

치환될 변수를 { 변수 } 로 묶어서 템플릿을 정의합니다.

In [8]:
from langchain_core.prompts import PromptTemplate

# template 정의. {country}는 변수로, 이후에 값이 들어갈 자리를 의미
template = "{DriverName}의 현재 F1 팀 소속은 어디인가요?"

# from_template 메소드를 이용하여 PromptTemplate 객체 생성
prompt = PromptTemplate.from_template(template)
prompt


PromptTemplate(input_variables=['DriverName'], input_types={}, partial_variables={}, template='{DriverName}의 현재 F1 팀 소속은 어디인가요?')

변수에 값을 넣어서 문장을 생성할 수 있습니다.

In [9]:
prompt = prompt.format(DriverName="Oliver Bearman")
prompt

'Oliver Bearman의 현재 F1 팀 소속은 어디인가요?'

In [10]:
# from_template 메소드를 이용하여 PromptTemplate 객체 생성
prompt = PromptTemplate.from_template(template)

# chain 생성
chain = prompt | llm

In [11]:
chain.invoke("Oliver Bearman").content

'Oliver Bearman은 현재 Red Bull Junior Team 소속입니다.'

## 방법 2. PromptTemplate 객체 생성과 동시에 prompt 생성

In [12]:
# template 정의
template = "{DriverName}의 현재 F1 팀 소속은 어디인가요?"

# PromptTemplate 객체를 활용하여 prompt_template 생성
prompt = PromptTemplate(
    template=template,
    input_variables=["DriverName"],
)

prompt


PromptTemplate(input_variables=['DriverName'], input_types={}, partial_variables={}, template='{DriverName}의 현재 F1 팀 소속은 어디인가요?')

In [13]:
# prompt 생성
prompt.format(DriverName="Oliver Bearman")

'Oliver Bearman의 현재 F1 팀 소속은 어디인가요?'

In [15]:
# template 정의
template = "{team_name1}과 {team_name2}의 소속 나라는 각각 어디인가요?"

# PromptTemplate 객체를 활용하여 prompt_template 생성
prompt = PromptTemplate(
    template=template,
    input_variables=["team_name1"],
    partial_variables={
        "team_name2": "Racing bulls"  # dictionary 형태로 partial_variables를 전달
    },
)

prompt

PromptTemplate(input_variables=['team_name1'], input_types={}, partial_variables={'team_name2': 'Racing bulls'}, template='{team_name1}과 {team_name2}의 소속 나라는 각각 어디인가요?')

In [16]:
prompt.format(team_name1="Haas MoneyGram F1")

'Haas MoneyGram F1과 Racing bulls의 소속 나라는 각각 어디인가요?'

In [17]:
prompt_partial = prompt.partial(team_name2="Ferrari")
prompt_partial

PromptTemplate(input_variables=['team_name1'], input_types={}, partial_variables={'team_name2': 'Ferrari'}, template='{team_name1}과 {team_name2}의 소속 나라는 각각 어디인가요?')

In [18]:
prompt_partial.format(team_name1="Haas MoneyGram F1")

'Haas MoneyGram F1과 Ferrari의 소속 나라는 각각 어디인가요?'

In [19]:
chain = prompt_partial | llm

In [22]:
chain.invoke("Haas MoneyGram F1").content

'Haas F1 팀은 미국을 대표하는 F1 팀이며, 소속 나라는 미국입니다. 반면에 Ferrari는 이탈리아를 대표하는 F1 팀으로, 소속 나라는 이탈리아입니다.'

In [23]:
chain.invoke({"team_name1" : "Haas MoneyGram F1", "team_name2" : "Ferrari",}).content

'Haas MoneyGram F1은 미국 소속이며, Ferrari는 이탈리아 소속입니다.'

### 날짜 예시

In [24]:
from datetime import datetime

# 오늘 날짜를 출력
datetime.now().strftime("%B %d")

'October 31'

In [25]:
# 날짜를 반환하는 함수 정의
def get_today():
    return datetime.now().strftime("%B %d")

In [46]:
prompt = PromptTemplate(
    template="오늘의 날짜는 {today} 입니다. 오늘의 날짜에 해당하는 달이 생일인 전직/현직 F1 드라이버 {n}명을 나열해 주세요. 생년월일을 표기해주세요. 없으면 없다고 해주세요.",
    input_variables=["n"],
    partial_variables={
        "today": get_today  # dictionary 형태로 partial_variables를 전달
    },
)

In [47]:
# prompt 생성
prompt.format(n=3)

'오늘의 날짜는 October 31 입니다. 오늘의 날짜에 해당하는 달이 생일인 전직/현직 F1 드라이버 3명을 나열해 주세요. 생년월일을 표기해주세요. 없으면 없다고 해주세요.'

In [48]:
# chain 을 생성합니다.
chain = prompt | llm

In [49]:
# chain 을 실행 후 결과를 확인합니다.
print(chain.invoke(3).content)

1. Jules Bianchi - October 3, 1989
2. Robert Doornbos - September 23, 1981
3. Gerhard Berger - August 27, 1959


In [50]:
# chain 을 실행 후 결과를 확인합니다.
print(chain.invoke({"today": "Oct 31", "n": 3}).content)

1. Lewis Hamilton - 1985년 1월 7일
2. Fernando Alonso - 1981년 7월 29일
3. Timo Glock - 1982년 3월 18일


-> 정확도가 그리 높지 않음

### 파일로부터 template 읽어오기

In [59]:
from langchain_core.prompts import load_prompt

prompt = load_prompt("week02_prompts/fruit_color.yaml", encoding="utf-8")
prompt

PromptTemplate(input_variables=['fruit'], input_types={}, partial_variables={}, template='{fruit}의 색깔이 뭐야?')

In [60]:
prompt.format(fruit="사과")

'사과의 색깔이 뭐야?'

In [63]:
prompt2 = load_prompt("week02_prompts/capital.yaml", encoding="utf-8")
print(prompt2.format(country="대한민국"))

대한민국의 수도에 대해서 알려주세요.
수도의 특징을 다음의 양식에 맞게 정리해 주세요.
300자 내외로 작성해 주세요.
한글로 작성해 주세요.
----
[양식]
1. 면적
2. 인구
3. 역사적 장소
4. 특산품

#Answer:



### ChatPromptTemplate

In [64]:
from langchain_core.prompts import ChatPromptTemplate

chat_prompt = ChatPromptTemplate.from_template("{country}의 수도는 어디인가요?")
chat_prompt

ChatPromptTemplate(input_variables=['country'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['country'], input_types={}, partial_variables={}, template='{country}의 수도는 어디인가요?'), additional_kwargs={})])

In [65]:
chat_prompt.format(country="대한민국")

'Human: 대한민국의 수도는 어디인가요?'

In [68]:
from langchain_core.prompts import ChatPromptTemplate

chat_template = ChatPromptTemplate.from_messages(
    [
        # role, message
        ("system", "당신은 Haas Moneygram F1 team 소속의 principal입니다. 당신의 이름은 {name} 입니다."),
        ("human", "반가워요!"),
        ("ai", "안녕하세요! 무엇을 도와드릴까요?"),
        ("human", "{user_input}"),
    ]
)

# 챗 message 를 생성합니다.
messages = chat_template.format_messages(
    name="테디", user_input="당신의 이름은 무엇입니까?"
)
messages

[SystemMessage(content='당신은 Haas Moneygram F1 team 소속의 principal입니다. 당신의 이름은 테디 입니다.', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='반가워요!', additional_kwargs={}, response_metadata={}),
 AIMessage(content='안녕하세요! 무엇을 도와드릴까요?', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='당신의 이름은 무엇입니까?', additional_kwargs={}, response_metadata={})]

In [69]:
llm.invoke(messages).content

'제 이름은 테디입니다. Haas Moneygram F1 team의 소속 principal 입니다.'

In [70]:
chain = chat_template | llm


In [71]:
chain.invoke({"name": "Teddy", "user_input": "당신의 이름은 무엇입니까?"}).content


'저는 Haas Moneygram F1 team의 principal인 Teddy입니다. 어떻게 도와드릴까요?'

### MessagePlaceholder

In [74]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

chat_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "당신은 요약 전문 AI 어시스턴트입니다. 당신의 임무는 주요 키워드로 대화를 요약하는 것입니다.",
        ),
        MessagesPlaceholder(variable_name="conversation"),
        ("human", "지금까지의 대화를 {word_count} 단어로 요약합니다."),
    ]
)
chat_prompt

ChatPromptTemplate(input_variables=['conversation', 'word_count'], input_types={'conversation': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessageChunk')], typing.Annotated[langchain_core.messages.human.HumanMessageChunk, Tag(tag='HumanMessageChunk')], typing.Annotated[langchain_core.messages.chat.ChatMessageChunk, Tag(tag='ChatMessageChunk')], typing.Annotated[langchain_core.messages.system.SystemMessageChunk, Tag(tag='SystemMessageChunk')], typing.Annota

In [75]:
formatted_chat_prompt = chat_prompt.format(
    word_count=5,
    conversation=[
        ("human", "안녕하세요! 저는 오늘 새로 입사한 테디 입니다. 만나서 반갑습니다."),
        ("ai", "반가워요! 앞으로 잘 부탁 드립니다."),
    ],
)

print(formatted_chat_prompt)

System: 당신은 요약 전문 AI 어시스턴트입니다. 당신의 임무는 주요 키워드로 대화를 요약하는 것입니다.
Human: 안녕하세요! 저는 오늘 새로 입사한 테디 입니다. 만나서 반갑습니다.
AI: 반가워요! 앞으로 잘 부탁 드립니다.
Human: 지금까지의 대화를 5 단어로 요약합니다.


In [80]:
chain = chat_prompt | llm | StrOutputParser()

In [81]:
# chain 실행 및 결과확인
chain.invoke(
    {
        "word_count": 5,
        "conversation": [
            (
                "human",
                "안녕하세요! 저는 오늘 새로 입사한 테디 입니다. 만나서 반갑습니다.",
            ),
            ("ai", "반가워요! 앞으로 잘 부탁 드립니다."),
        ],
    }
)

'테디 새로 입사, 만남 인사.'

# 02. 퓨샷 프롬프트(FewShotPromptTemplate)

In [82]:
from dotenv import load_dotenv

load_dotenv()

True

In [85]:
# LangSmith 추적을 설정합니다. https://smith.langchain.com
# !pip install -qU langchain-teddynote
from langchain_teddynote import logging

# 프로젝트 이름을 입력합니다.
logging.langsmith("CH02-Prompt-Junu")

LangSmith 추적을 시작합니다.
[프로젝트명]
CH02-Prompt-Junu


## FewShotPromptTemplate

In [88]:
from langchain_openai import ChatOpenAI
from langchain_teddynote.messages import stream_response

# 객체 생성
llm = ChatOpenAI(
    temperature=0,  # 창의성
    model_name="gpt-4-turbo",  # 모델명
)

# 질의내용
question = "현재 가장 위대한 F1 팀이 뭐야?"

# 질의
answer = llm.stream(question)
stream_response(answer)

2023년 현재, 포뮬러 원(F1)에서 가장 성공적인 팀을 꼽으라면 여러 요소를 고려해야 합니다. 최근 몇 년간의 성적, 월드 챔피언십 타이틀 수, 그리고 현재 시즌의 성적 등이 중요한 기준이 될 수 있습니다.

최근 몇 년간을 보면, 메르세데스-AMG 페트로나스 포뮬러 원 팀이 매우 강력한 성적을 보여주었습니다. 2014년부터 2020년까지 연속으로 컨스트럭터스 챔피언십(팀 챔피언십)을 획득하며 독보적인 성과를 이루었습니다. 또한, 이 기간 동안 루이스 해밀턴과 니코 로즈버그 같은 드라이버들이 드라이버 챔피언십에서도 여러 번 우승을 차지했습니다.

그러나 2021년과 2022년에는 레드불 레이싱이 강세를 보이며 맥스 페르스타펜이 드라이버 챔피언십에서 우승을 차지했고, 팀도 컨스트럭터스 챔피언십에서 좋은 성적을 거두었습니다.

따라서, 현재 가장 위대한 F1 팀을 선택하는 것은 최근의 경향과 성적에 따라 달라질 수 있으며, 메르세데스와 레드불 두 팀 모두 최근 몇 년간 F1에서 매우 강력한 경쟁력을 보여주고 있습니다.

In [89]:
from langchain_core.prompts.few_shot import FewShotPromptTemplate
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser


examples = [
    {
        "question": "스티브 잡스와 아인슈타인 중 누가 더 오래 살았나요?",
        "answer": """이 질문에 추가 질문이 필요한가요: 예.
추가 질문: 스티브 잡스는 몇 살에 사망했나요?
중간 답변: 스티브 잡스는 56세에 사망했습니다.
추가 질문: 아인슈타인은 몇 살에 사망했나요?
중간 답변: 아인슈타인은 76세에 사망했습니다.
최종 답변은: 아인슈타인
""",
    },
    {
        "question": "네이버의 창립자는 언제 태어났나요?",
        "answer": """이 질문에 추가 질문이 필요한가요: 예.
추가 질문: 네이버의 창립자는 누구인가요?
중간 답변: 네이버는 이해진에 의해 창립되었습니다.
추가 질문: 이해진은 언제 태어났나요?
중간 답변: 이해진은 1967년 6월 22일에 태어났습니다.
최종 답변은: 1967년 6월 22일
""",
    },
    {
        "question": "율곡 이이의 어머니가 태어난 해의 통치하던 왕은 누구인가요?",
        "answer": """이 질문에 추가 질문이 필요한가요: 예.
추가 질문: 율곡 이이의 어머니는 누구인가요?
중간 답변: 율곡 이이의 어머니는 신사임당입니다.
추가 질문: 신사임당은 언제 태어났나요?
중간 답변: 신사임당은 1504년에 태어났습니다.
추가 질문: 1504년에 조선을 통치한 왕은 누구인가요?
중간 답변: 1504년에 조선을 통치한 왕은 연산군입니다.
최종 답변은: 연산군
""",
    },
    {
        "question": "올드보이와 기생충의 감독이 같은 나라 출신인가요?",
        "answer": """이 질문에 추가 질문이 필요한가요: 예.
추가 질문: 올드보이의 감독은 누구인가요?
중간 답변: 올드보이의 감독은 박찬욱입니다.
추가 질문: 박찬욱은 어느 나라 출신인가요?
중간 답변: 박찬욱은 대한민국 출신입니다.
추가 질문: 기생충의 감독은 누구인가요?
중간 답변: 기생충의 감독은 봉준호입니다.
추가 질문: 봉준호는 어느 나라 출신인가요?
중간 답변: 봉준호는 대한민국 출신입니다.
최종 답변은: 예
""",
    },
]


FewShotPromptTemplate 없이는 직접 형식을 지정한뒤 입력해야한다.

In [95]:
example_prompt = PromptTemplate.from_template(
    "Question:\n{question}\nAnswer:\n{answer}"
)

print(example_prompt.format(**examples[0]))


Question:
스티브 잡스와 아인슈타인 중 누가 더 오래 살았나요?
Answer:
이 질문에 추가 질문이 필요한가요: 예.
추가 질문: 스티브 잡스는 몇 살에 사망했나요?
중간 답변: 스티브 잡스는 56세에 사망했습니다.
추가 질문: 아인슈타인은 몇 살에 사망했나요?
중간 답변: 아인슈타인은 76세에 사망했습니다.
최종 답변은: 아인슈타인



FewShotPromptTemplate을 사용한다면 간단하게 suffix parameter를 통해 examples를 dictionary 형태로 넣기만 해도 된다

In [92]:
prompt = FewShotPromptTemplate(
    examples=examples,
    example_prompt=example_prompt,
    suffix="Question:\n{question}\nAnswer:",
    input_variables=["question"],
)

question = "Google이 창립된 연도에 Bill Gates의 나이는 몇 살인가요?"
final_prompt = prompt.format(question=question)
print(final_prompt)

Question:
스티브 잡스와 아인슈타인 중 누가 더 오래 살았나요?
Answer:
이 질문에 추가 질문이 필요한가요: 예.
추가 질문: 스티브 잡스는 몇 살에 사망했나요?
중간 답변: 스티브 잡스는 56세에 사망했습니다.
추가 질문: 아인슈타인은 몇 살에 사망했나요?
중간 답변: 아인슈타인은 76세에 사망했습니다.
최종 답변은: 아인슈타인


Question:
네이버의 창립자는 언제 태어났나요?
Answer:
이 질문에 추가 질문이 필요한가요: 예.
추가 질문: 네이버의 창립자는 누구인가요?
중간 답변: 네이버는 이해진에 의해 창립되었습니다.
추가 질문: 이해진은 언제 태어났나요?
중간 답변: 이해진은 1967년 6월 22일에 태어났습니다.
최종 답변은: 1967년 6월 22일


Question:
율곡 이이의 어머니가 태어난 해의 통치하던 왕은 누구인가요?
Answer:
이 질문에 추가 질문이 필요한가요: 예.
추가 질문: 율곡 이이의 어머니는 누구인가요?
중간 답변: 율곡 이이의 어머니는 신사임당입니다.
추가 질문: 신사임당은 언제 태어났나요?
중간 답변: 신사임당은 1504년에 태어났습니다.
추가 질문: 1504년에 조선을 통치한 왕은 누구인가요?
중간 답변: 1504년에 조선을 통치한 왕은 연산군입니다.
최종 답변은: 연산군


Question:
올드보이와 기생충의 감독이 같은 나라 출신인가요?
Answer:
이 질문에 추가 질문이 필요한가요: 예.
추가 질문: 올드보이의 감독은 누구인가요?
중간 답변: 올드보이의 감독은 박찬욱입니다.
추가 질문: 박찬욱은 어느 나라 출신인가요?
중간 답변: 박찬욱은 대한민국 출신입니다.
추가 질문: 기생충의 감독은 누구인가요?
중간 답변: 기생충의 감독은 봉준호입니다.
추가 질문: 봉준호는 어느 나라 출신인가요?
중간 답변: 봉준호는 대한민국 출신입니다.
최종 답변은: 예


Question:
Google이 창립된 연도에 Bill Gates의 나이는 몇 살인가요?
Answer:


In [93]:
# 결과 출력
answer = llm.stream(final_prompt)
stream_response(answer)

이 질문에 추가 질문이 필요한가요: 예.
추가 질문: Google은 언제 창립되었나요?
중간 답변: Google은 1998년에 창립되었습니다.
추가 질문: Bill Gates는 언제 태어났나요?
중간 답변: Bill Gates는 1955년 10월 28일에 태어났습니다.
최종 답변은: 1998년에 Bill Gates의 나이는 43세입니다.

In [94]:
prompt = FewShotPromptTemplate(
    examples=examples,
    example_prompt=example_prompt,
    suffix="Question:\n{question}\nAnswer:",
    input_variables=["question"],
)

# chain 생성
chain = prompt | llm | StrOutputParser()

# 결과 출력
answer = chain.stream(
    {"question": "Google이 창립된 연도에 Bill Gates의 나이는 몇 살인가요?"}
)
stream_response(answer)

이 질문에 추가 질문이 필요한가요: 예.
추가 질문: Google은 언제 창립되었나요?
중간 답변: Google은 1998년에 창립되었습니다.
추가 질문: Bill Gates는 언제 태어났나요?
중간 답변: Bill Gates는 1955년 10월 28일에 태어났습니다.
최종 답변은: 1998년에 Bill Gates의 나이는 43세입니다.

Example Selector
예제가 많은 경우 프롬프트에 포함할 예제를 선택해야 할 수도 있습니다. Example Selector 는 이 작업을 담당하는 클래스입니다.

In [98]:
from langchain_core.example_selectors import (
    MaxMarginalRelevanceExampleSelector,
    SemanticSimilarityExampleSelector,
)
from langchain_openai import OpenAIEmbeddings
from langchain_chroma import Chroma

# Vector DB 생성 (저장소 이름, 임베딩 클래스)
chroma = Chroma("example_selector", OpenAIEmbeddings())

example_selector = SemanticSimilarityExampleSelector.from_examples(
    # 여기에는 선택 가능한 예시 목록이 있습니다.
    examples,
    # 여기에는 의미적 유사성을 측정하는 데 사용되는 임베딩을 생성하는 임베딩 클래스가 있습니다.
    OpenAIEmbeddings(),
    # 여기에는 임베딩을 저장하고 유사성 검색을 수행하는 데 사용되는 VectorStore 클래스가 있습니다.
    Chroma,
    # 이것은 생성할 예시의 수입니다.
    k=1,
)

# 입력과 가장 유사한 예시를 선택합니다.
selected_examples = example_selector.select_examples({"question": question})

question = "Google이 창립된 연도에 Bill Gates의 나이는 몇 살인가요?"
print(f"입력에 가장 유사한 예시:\n{question}\n")
for example in selected_examples:
    print(f'question:\n{example["question"]}')
    print(f'answer:\n{example["answer"]}')


Failed to send telemetry event ClientStartEvent: capture() takes 1 positional argument but 3 were given
Failed to send telemetry event ClientCreateCollectionEvent: capture() takes 1 positional argument but 3 were given
Failed to send telemetry event ClientStartEvent: capture() takes 1 positional argument but 3 were given
Failed to send telemetry event ClientCreateCollectionEvent: capture() takes 1 positional argument but 3 were given


입력에 가장 유사한 예시:
Google이 창립된 연도에 Bill Gates의 나이는 몇 살인가요?

question:
네이버의 창립자는 언제 태어났나요?
answer:
이 질문에 추가 질문이 필요한가요: 예.
추가 질문: 네이버의 창립자는 누구인가요?
중간 답변: 네이버는 이해진에 의해 창립되었습니다.
추가 질문: 이해진은 언제 태어났나요?
중간 답변: 이해진은 1967년 6월 22일에 태어났습니다.
최종 답변은: 1967년 6월 22일



In [99]:
prompt = FewShotPromptTemplate(
    example_selector=example_selector,
    example_prompt=example_prompt,
    suffix="Question:\n{question}\nAnswer:",
    input_variables=["question"],
)

question = "Google이 창립된 연도에 Bill Gates의 나이는 몇 살인가요?"
example_selector_prompt = prompt.format(question=question)
print(example_selector_prompt)


Question:
네이버의 창립자는 언제 태어났나요?
Answer:
이 질문에 추가 질문이 필요한가요: 예.
추가 질문: 네이버의 창립자는 누구인가요?
중간 답변: 네이버는 이해진에 의해 창립되었습니다.
추가 질문: 이해진은 언제 태어났나요?
중간 답변: 이해진은 1967년 6월 22일에 태어났습니다.
최종 답변은: 1967년 6월 22일


Question:
Google이 창립된 연도에 Bill Gates의 나이는 몇 살인가요?
Answer:


이번에는 ExampleSelector 를 사용하여 FewShotPromptTemplate 을 생성합니다.

In [100]:
prompt = FewShotPromptTemplate(
    example_selector=example_selector,
    example_prompt=example_prompt,
    suffix="Question:\n{question}\nAnswer:",
    input_variables=["question"],
)

# 체인 생성
chain = prompt | llm

In [101]:
# 결과 출력
answer = chain.stream(
    {"question": "Google이 창립된 연도에 Bill Gates의 나이는 몇 살인가요?"}
)
stream_response(answer)

이 질문에 추가 질문이 필요한가요: 예.
추가 질문: Google이 언제 창립되었나요?
중간 답변: Google은 1998년에 창립되었습니다.
추가 질문: Bill Gates는 언제 태어났나요?
중간 답변: Bill Gates는 1955년 10월 28일에 태어났습니다.
최종 계산: 1998년에서 1955년을 빼면 43년이므로, Bill Gates는 Google이 창립된 해에 43세였습니다.
최종 답변은: 43세

## FewShotChatMessagePromptTemplate

In [102]:
examples = [
    {
        "instruction": "당신은 회의록 작성 전문가 입니다. 주어진 정보를 바탕으로 회의록을 작성해 주세요",
        "input": "2023년 12월 25일, XYZ 회사의 마케팅 전략 회의가 오후 3시에 시작되었다. 회의에는 마케팅 팀장인 김수진, 디지털 마케팅 담당자인 박지민, 소셜 미디어 관리자인 이준호가 참석했다. 회의의 주요 목적은 2024년 상반기 마케팅 전략을 수립하고, 새로운 소셜 미디어 캠페인에 대한 아이디어를 논의하는 것이었다. 팀장인 김수진은 최근 시장 동향에 대한 간략한 개요를 제공했으며, 이어서 각 팀원이 자신의 분야에서의 전략적 아이디어를 발표했다.",
        "answer": """
회의록: XYZ 회사 마케팅 전략 회의
일시: 2023년 12월 25일
장소: XYZ 회사 회의실
참석자: 김수진 (마케팅 팀장), 박지민 (디지털 마케팅 담당자), 이준호 (소셜 미디어 관리자)

1. 개회
   - 회의는 김수진 팀장의 개회사로 시작됨.
   - 회의의 목적은 2024년 상반기 마케팅 전략 수립 및 새로운 소셜 미디어 캠페인 아이디어 논의.

2. 시장 동향 개요 (김수진)
   - 김수진 팀장은 최근 시장 동향에 대한 분석을 제시.
   - 소비자 행동 변화와 경쟁사 전략에 대한 통찰 공유.

3. 디지털 마케팅 전략 (박지민)
   - 박지민은 디지털 마케팅 전략에 대해 발표.
   - 온라인 광고와 SEO 최적화 방안에 중점을 둠.

4. 소셜 미디어 캠페인 (이준호)
   - 이준호는 새로운 소셜 미디어 캠페인에 대한 아이디어를 제안.
   - 인플루언서 마케팅과 콘텐츠 전략에 대한 계획을 설명함.

5. 종합 논의
   - 팀원들 간의 아이디어 공유 및 토론.
   - 각 전략에 대한 예산 및 자원 배분에 대해 논의.

6. 마무리
   - 다음 회의 날짜 및 시간 확정.
   - 회의록 정리 및 배포는 박지민 담당.
""",
    },
    {
        "instruction": "당신은 요약 전문가 입니다. 다음 주어진 정보를 바탕으로 내용을 요약해 주세요",
        "input": "이 문서는 '지속 가능한 도시 개발을 위한 전략'에 대한 20페이지 분량의 보고서입니다. 보고서는 지속 가능한 도시 개발의 중요성, 현재 도시화의 문제점, 그리고 도시 개발을 지속 가능하게 만들기 위한 다양한 전략을 포괄적으로 다루고 있습니다. 이 보고서는 또한 성공적인 지속 가능한 도시 개발 사례를 여러 국가에서 소개하고, 이러한 사례들을 통해 얻은 교훈을 요약하고 있습니다.",
        "answer": """
문서 요약: 지속 가능한 도시 개발을 위한 전략 보고서

- 중요성: 지속 가능한 도시 개발이 필수적인 이유와 그에 따른 사회적, 경제적, 환경적 이익을 강조.
- 현 문제점: 현재의 도시화 과정에서 발생하는 주요 문제점들, 예를 들어 환경 오염, 자원 고갈, 불평등 증가 등을 분석.
- 전략: 지속 가능한 도시 개발을 달성하기 위한 다양한 전략 제시. 이에는 친환경 건축, 대중교통 개선, 에너지 효율성 증대, 지역사회 참여 강화 등이 포함됨.
- 사례 연구: 전 세계 여러 도시의 성공적인 지속 가능한 개발 사례를 소개. 예를 들어, 덴마크의 코펜하겐, 일본의 요코하마 등의 사례를 통해 실현 가능한 전략들을 설명.
- 교훈: 이러한 사례들에서 얻은 주요 교훈을 요약. 강조된 교훈에는 다각적 접근의 중요성, 지역사회와의 협력, 장기적 계획의 필요성 등이 포함됨.

이 보고서는 지속 가능한 도시 개발이 어떻게 현실적이고 효과적인 형태로 이루어질 수 있는지에 대한 심도 있는 분석을 제공합니다.
""",
    },
    {
        "instruction": "당신은 문장 교정 전문가 입니다. 다음 주어진 문장을 교정해 주세요",
        "input": "우리 회사는 새로운 마케팅 전략을 도입하려고 한다. 이를 통해 고객과의 소통이 더 효과적이 될 것이다.",
        "answer": "본 회사는 새로운 마케팅 전략을 도입함으로써, 고객과의 소통을 보다 효과적으로 개선할 수 있을 것으로 기대된다.",
    },
]


In [103]:
from langchain_core.prompts import ChatPromptTemplate, FewShotChatMessagePromptTemplate
from langchain_core.example_selectors import (
    SemanticSimilarityExampleSelector,
)
from langchain_openai import OpenAIEmbeddings
from langchain_chroma import Chroma

chroma = Chroma("fewshot_chat", OpenAIEmbeddings())

example_prompt = ChatPromptTemplate.from_messages(
    [
        ("human", "{instruction}:\n{input}"),
        ("ai", "{answer}"),
    ]
)

example_selector = SemanticSimilarityExampleSelector.from_examples(
    # 여기에는 선택 가능한 예시 목록이 있습니다.
    examples,
    # 여기에는 의미적 유사성을 측정하는 데 사용되는 임베딩을 생성하는 임베딩 클래스가 있습니다.
    OpenAIEmbeddings(),
    # 여기에는 임베딩을 저장하고 유사성 검색을 수행하는 데 사용되는 VectorStore 클래스가 있습니다.
    chroma,
    # 이것은 생성할 예시의 수입니다.
    k=1,
)

few_shot_prompt = FewShotChatMessagePromptTemplate(
    example_selector=example_selector,
    example_prompt=example_prompt,
)


Failed to send telemetry event ClientStartEvent: capture() takes 1 positional argument but 3 were given
Failed to send telemetry event ClientCreateCollectionEvent: capture() takes 1 positional argument but 3 were given
Failed to send telemetry event ClientStartEvent: capture() takes 1 positional argument but 3 were given
Failed to send telemetry event ClientCreateCollectionEvent: capture() takes 1 positional argument but 3 were given


In [104]:
question = {
    "instruction": "회의록을 작성해 주세요",
    "input": "2023년 12월 26일, ABC 기술 회사의 제품 개발 팀은 새로운 모바일 애플리케이션 프로젝트에 대한 주간 진행 상황 회의를 가졌다. 이 회의에는 프로젝트 매니저인 최현수, 주요 개발자인 황지연, UI/UX 디자이너인 김태영이 참석했다. 회의의 주요 목적은 프로젝트의 현재 진행 상황을 검토하고, 다가오는 마일스톤에 대한 계획을 수립하는 것이었다. 각 팀원은 자신의 작업 영역에 대한 업데이트를 제공했고, 팀은 다음 주까지의 목표를 설정했다.",
}

example_selector.select_examples(question)

[{'answer': '\n회의록: XYZ 회사 마케팅 전략 회의\n일시: 2023년 12월 25일\n장소: XYZ 회사 회의실\n참석자: 김수진 (마케팅 팀장), 박지민 (디지털 마케팅 담당자), 이준호 (소셜 미디어 관리자)\n\n1. 개회\n   - 회의는 김수진 팀장의 개회사로 시작됨.\n   - 회의의 목적은 2024년 상반기 마케팅 전략 수립 및 새로운 소셜 미디어 캠페인 아이디어 논의.\n\n2. 시장 동향 개요 (김수진)\n   - 김수진 팀장은 최근 시장 동향에 대한 분석을 제시.\n   - 소비자 행동 변화와 경쟁사 전략에 대한 통찰 공유.\n\n3. 디지털 마케팅 전략 (박지민)\n   - 박지민은 디지털 마케팅 전략에 대해 발표.\n   - 온라인 광고와 SEO 최적화 방안에 중점을 둠.\n\n4. 소셜 미디어 캠페인 (이준호)\n   - 이준호는 새로운 소셜 미디어 캠페인에 대한 아이디어를 제안.\n   - 인플루언서 마케팅과 콘텐츠 전략에 대한 계획을 설명함.\n\n5. 종합 논의\n   - 팀원들 간의 아이디어 공유 및 토론.\n   - 각 전략에 대한 예산 및 자원 배분에 대해 논의.\n\n6. 마무리\n   - 다음 회의 날짜 및 시간 확정.\n   - 회의록 정리 및 배포는 박지민 담당.\n',
  'input': '2023년 12월 25일, XYZ 회사의 마케팅 전략 회의가 오후 3시에 시작되었다. 회의에는 마케팅 팀장인 김수진, 디지털 마케팅 담당자인 박지민, 소셜 미디어 관리자인 이준호가 참석했다. 회의의 주요 목적은 2024년 상반기 마케팅 전략을 수립하고, 새로운 소셜 미디어 캠페인에 대한 아이디어를 논의하는 것이었다. 팀장인 김수진은 최근 시장 동향에 대한 간략한 개요를 제공했으며, 이어서 각 팀원이 자신의 분야에서의 전략적 아이디어를 발표했다.',
  'instruction': '당신은 회의록 작성 전문가 입니다. 주어진 정보를 바탕으로 회의록을 작성해 주세요'}]

In [105]:
final_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant.",
        ),
        few_shot_prompt,
        ("human", "{instruction}\n{input}"),
    ]
)


In [106]:
# chain 생성
chain = final_prompt | llm

# 실행 및 결과 출력
answer = chain.stream(question)
stream_response(answer)

회의록: ABC 기술 회사 제품 개발 팀 주간 진행 상황 회의
일시: 2023년 12월 26일
장소: ABC 기술 회사 회의실
참석자: 최현수 (프로젝트 매니저), 황지연 (주요 개발자), 김태영 (UI/UX 디자이너)

1. 개회
   - 최현수 프로젝트 매니저가 회의를 개회하고, 회의의 목적을 설명함.
   - 목적: 프로젝트의 현재 진행 상황 검토 및 다가오는 마일스톤에 대한 계획 수립.

2. 프로젝트 진행 상황 보고
   - 황지연 개발자가 개발 진행 상황에 대해 보고.
     - 현재 개발 중인 기능들의 상태와 최근 해결된 문제점들을 공유.
     - 다음 주까지 완료 목표로 설정한 기능들을 명시.
   - 김태영 디자이너가 UI/UX 디자인 진행 상황에 대해 보고.
     - 최근 완성된 디자인 요소들과 사용자 피드백을 바탕으로 한 개선 사항을 공유.
     - 다음 단계의 디자인 작업 계획을 설명.

3. 마일스톤 계획
   - 최현수가 다가오는 마일스톤에 대한 전체적인 계획을 제시.
     - 중요한 날짜와 목표들을 확인하고, 각 팀원의 역할과 기대 결과를 명확히 함.
   - 팀원들이 각자의 업무에 대해 피드백을 주고받음.

4. 다음 주 목표 설정
   - 각 팀원이 다음 주까지의 구체적인 목표를 설정.
     - 황지연은 특정 기능 개발 완료.
     - 김태영은 사용자 인터페이스의 특정 부분 디자인 완성.

5. 마무리
   - 다음 회의 날짜 및 시간 확정.
   - 회의록 정리 및 배포는 최현수 매니저가 담당.

회의는 모든 안건이 충분히 논의되고 각 팀원의 다음 주 목표가 명확히 설정된 후 종료됨.

## Example Selector 의 유사도 검색 문제 해결

유사도 계산시 instruction 과 input 을 사용하고 있습니다. 하지만, instruction 만 사용하여 검색시 제대로된 유사도 결과가 나오지 않습니다.

이를 해결하기 위해 커스텀 유사도 계산을 위한 클래스를 정의합니다. 아래는 잘못 검색된 결과의 예시입니다.

In [107]:
question = {
    "instruction": "회의록을 작성해 주세요",
}

example_selector.select_examples(question)


[{'answer': '본 회사는 새로운 마케팅 전략을 도입함으로써, 고객과의 소통을 보다 효과적으로 개선할 수 있을 것으로 기대된다.',
  'input': '우리 회사는 새로운 마케팅 전략을 도입하려고 한다. 이를 통해 고객과의 소통이 더 효과적이 될 것이다.',
  'instruction': '당신은 문장 교정 전문가 입니다. 다음 주어진 문장을 교정해 주세요'}]

In [108]:
# 커스텀 하지 않은 기본 예제 선택기를 사용했을 때 결과
example_selector.select_examples({"instruction": "다음 문장을 요약해 주세요"})


[{'answer': '본 회사는 새로운 마케팅 전략을 도입함으로써, 고객과의 소통을 보다 효과적으로 개선할 수 있을 것으로 기대된다.',
  'input': '우리 회사는 새로운 마케팅 전략을 도입하려고 한다. 이를 통해 고객과의 소통이 더 효과적이 될 것이다.',
  'instruction': '당신은 문장 교정 전문가 입니다. 다음 주어진 문장을 교정해 주세요'}]

In [109]:
from langchain_teddynote.prompts import CustomExampleSelector

# 커스텀 예제 선택기 생성
custom_selector = CustomExampleSelector(examples, OpenAIEmbeddings())

# 커스텀 예제 선택기를 사용했을 때 결과
custom_selector.select_examples({"instruction": "다음 문장을 교정 작성해 주세요"})

[{'instruction': '당신은 문장 교정 전문가 입니다. 다음 주어진 문장을 교정해 주세요',
  'input': '우리 회사는 새로운 마케팅 전략을 도입하려고 한다. 이를 통해 고객과의 소통이 더 효과적이 될 것이다.',
  'answer': '본 회사는 새로운 마케팅 전략을 도입함으로써, 고객과의 소통을 보다 효과적으로 개선할 수 있을 것으로 기대된다.'}]

In [110]:
example_prompt = ChatPromptTemplate.from_messages(
    [
        ("human", "{instruction}:\n{input}"),
        ("ai", "{answer}"),
    ]
)

custom_fewshot_prompt = FewShotChatMessagePromptTemplate(
    example_selector=custom_selector,  # 커스텀 예제 선택기 사용
    example_prompt=example_prompt,  # 예제 프롬프트 사용
)

custom_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant.",
        ),
        custom_fewshot_prompt,
        ("human", "{instruction}\n{input}"),
    ]
)


In [111]:
# chain 을 생성합니다.
chain = custom_prompt | llm


In [113]:
question = {
    "instruction": "회의록을 작성해 주세요",
    "input": "2023년 12월 26일, ABC 기술 회사의 제품 개발 팀은 새로운 모바일 애플리케이션 프로젝트에 대한 주간 진행 상황 회의를 가졌다. 이 회의에는 프로젝트 매니저인 최현수, 주요 개발자인 황지연, UI/UX 디자이너인 김태영이 참석했다. 회의의 주요 목적은 프로젝트의 현재 진행 상황을 검토하고, 다가오는 마일스톤에 대한 계획을 수립하는 것이었다. 각 팀원은 자신의 작업 영역에 대한 업데이트를 제공했고, 팀은 다음 주까지의 목표를 설정했다.",
}

# 실행 및 결과 출력
stream_response(chain.stream(question))


회의록: ABC 기술 회사 제품 개발 팀 주간 진행 상황 회의
일시: 2023년 12월 26일
장소: ABC 기술 회사 회의실
참석자: 최현수 (프로젝트 매니저), 황지연 (주요 개발자), 김태영 (UI/UX 디자이너)

1. 개회
   - 최현수 프로젝트 매니저가 회의를 개회하고, 회의의 목적을 설명함.
   - 목적: 프로젝트의 현재 진행 상황 검토 및 다가오는 마일스톤에 대한 계획 수립.

2. 프로젝트 진행 상황 보고
   - 황지연 개발자가 개발 진행 상황에 대해 보고.
     - 현재 개발 중인 기능들의 진행 상태와 예상 완료 시점 공유.
     - 최근 해결된 주요 버그와 남아있는 이슈에 대해 논의.
   - 김태영 디자이너가 UI/UX 디자인 진행 상황에 대해 보고.
     - 새로운 디자인 요소의 구현 상태와 사용자 피드백에 대한 개요 제공.
     - 다음 단계의 디자인 개선 계획 발표.

3. 다가오는 마일스톤 계획
   - 최현수가 다음 주까지의 목표와 마일스톤을 설정.
     - 개발 팀은 주요 기능의 베타 버전 완성을 목표로 함.
     - 디자인 팀은 최종 사용자 테스트를 위한 디자인 수정을 완료할 계획.

4. 문제점 및 해결 방안 논의
   - 팀원들이 현재 직면한 문제점들을 공유하고 해결 방안을 모색.
   - 자원 배분과 일정 조정에 대한 필요성 논의.

5. 다음 회의 일정
   - 다음 주간 진행 상황 회의는 다음 주 월요일 같은 시간에 진행하기로 합의.

6. 마무리
   - 최현수 매니저가 회의를 마무리하며, 모든 팀원에게 각자의 업무에 최선을 다할 것을 당부.

회의록 작성자: 최현수
회의록 승인: 최현수 (프로젝트 매니저)

In [114]:
question = {
    "instruction": "문서를 요약해 주세요",
    "input": "이 문서는 '2023년 글로벌 경제 전망'에 관한 30페이지에 달하는 상세한 보고서입니다. 보고서는 세계 경제의 현재 상태, 주요 국가들의 경제 성장률, 글로벌 무역 동향, 그리고 다가오는 해에 대한 경제 예측을 다룹니다. 이 보고서는 또한 다양한 경제적, 정치적, 환경적 요인들이 세계 경제에 미칠 영향을 분석하고 있습니다.",
}

# 실행 및 결과 출력
stream_response(chain.stream(question))


문서 요약: 2023년 글로벌 경제 전망 보고서

- 현재 상태: 세계 경제의 현재 상황에 대한 분석을 제공하며, 경제 회복의 진행 상황과 주요 도전 과제를 설명.
- 경제 성장률: 주요 국가들의 경제 성장률을 검토하고, 각 국가의 경제 상황에 대한 상세한 정보를 제공.
- 글로벌 무역: 현재의 글로벌 무역 동향을 분석하고, 무역 정책 변화와 그 영향을 평가.
- 경제 예측: 2023년에 대한 경제 예측을 제시하며, 성장 전망과 가능한 위험 요소들을 고려.
- 영향 분석: 경제적, 정치적, 환경적 요인들이 세계 경제에 미칠 영향을 심층 분석.

이 보고서는 다가오는 해의 글로벌 경제 상황에 대한 포괄적인 이해를 돕기 위해 다양한 요인들을 종합적으로 검토하고 있습니다.

In [115]:
question = {
    "instruction": "문장을 교정해 주세요",
    "input": "회사는 올해 매출이 증가할 것으로 예상한다. 새로운 전략이 잘 작동하고 있다.",
}

# 실행 및 결과 출력
stream_response(chain.stream(question))


회사는 올해 매출이 증가할 것으로 예상하고 있다. 새로운 전략이 효과적으로 작동하고 있기 때문이다.

# 03. LangChain Hub

In [116]:
from langchain import hub

# 가장 최신 버전의 프롬프트를 가져옵니다.
prompt = hub.pull("rlm/rag-prompt")


In [117]:
# 프롬프트 내용 출력
print(prompt)


input_variables=['context', 'question'] input_types={} partial_variables={} metadata={'lc_hub_owner': 'rlm', 'lc_hub_repo': 'rag-prompt', 'lc_hub_commit_hash': '50442af133e61576e74536c6556cefe1fac147cad032f4377b60c436e6cdcb6e'} messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, template="You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.\nQuestion: {question} \nContext: {context} \nAnswer:"), additional_kwargs={})]


In [118]:
# 특정 버전의 프롬프트를 가져오려면 버전 해시를 지정하세요
prompt = hub.pull("rlm/rag-prompt:50442af1")
prompt


ChatPromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, metadata={'lc_hub_owner': 'rlm', 'lc_hub_repo': 'rag-prompt', 'lc_hub_commit_hash': '50442af133e61576e74536c6556cefe1fac147cad032f4377b60c436e6cdcb6e'}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, template="You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.\nQuestion: {question} \nContext: {context} \nAnswer:"), additional_kwargs={})])

## Prompt Hub 에 자신의 프롬프트 등록

In [120]:
from langchain.prompts import ChatPromptTemplate


prompt = ChatPromptTemplate.from_template(
    "주어진 내용을 바탕으로 F1에 대한 전반적인 지식을 바탕으로 내용을 비유하고 구조화하세요. 답변은 반드시 한글로 작성하되 고유명사는 영어를 허용합니다.\n\nCONTEXT: {context}\n\nSUMMARY:"
)
prompt

ChatPromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, template='주어진 내용을 바탕으로 F1에 대한 전반적인 지식을 바탕으로 내용을 비유하고 구조화하세요. 답변은 반드시 한글로 작성하되 고유명사는 영어를 허용합니다.\n\nCONTEXT: {context}\n\nSUMMARY:'), additional_kwargs={})])

In [127]:
from langchain import hub

# 프롬프트를 허브에 업로드합니다.
hub.push("simple-summary-korean-f1", prompt)

'https://smith.langchain.com/prompts/simple-summary-korean-f1/b009d115?organizationId=9437d3d5-0cd1-400d-8617-92cc576bb86a'

In [128]:
from langchain import hub

# 프롬프트를 허브로부터 가져옵니다.
pulled_prompt = hub.pull("simple-summary-korean-f1")

In [129]:
# 프롬프트 내용 출력
print(pulled_prompt)

input_variables=['context'] input_types={} partial_variables={} metadata={'lc_hub_owner': '-', 'lc_hub_repo': 'simple-summary-korean-f1', 'lc_hub_commit_hash': 'b009d115a13fd73945432957ca54a886bb2426e0f9dd3c3d8d0cbd37a049eff8'} messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, template='주어진 내용을 바탕으로 F1에 대한 전반적인 지식을 바탕으로 내용을 비유하고 구조화하세요. 답변은 반드시 한글로 작성하되 고유명사는 영어를 허용합니다.\n\nCONTEXT: {context}\n\nSUMMARY:'), additional_kwargs={})]


# 04. 개인화된 프롬프트(Hub에 업로드)

In [131]:
# Owner 지정
PROMPT_OWNER = "junu"

In [132]:
from langchain import hub
from langchain.prompts import PromptTemplate

prompt_title = "summary-stuff-documents"

# 요약문을 작성하기 위한 프롬프트 정의 (직접 프롬프트를 작성하는 경우)
prompt_template = """Please summarize the sentence according to the following REQUEST.
REQUEST:
1. Summarize the main points in bullet points.
2. Each summarized sentence must start with an emoji that fits the meaning of the each sentence.
3. Use various emojis to make the summary more interesting.
4. DO NOT include any unnecessary information.

CONTEXT:
{context}

SUMMARY:"
"""
prompt = PromptTemplate.from_template(prompt_template)
prompt


PromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, template='Please summarize the sentence according to the following REQUEST.\nREQUEST:\n1. Summarize the main points in bullet points.\n2. Each summarized sentence must start with an emoji that fits the meaning of the each sentence.\n3. Use various emojis to make the summary more interesting.\n4. DO NOT include any unnecessary information.\n\nCONTEXT:\n{context}\n\nSUMMARY:"\n')

In [134]:
hub.push(f"{prompt_title}", prompt)

'https://smith.langchain.com/prompts/summary-stuff-documents/ccf1dc3e?organizationId=9437d3d5-0cd1-400d-8617-92cc576bb86a'

## Map Prompt

In [135]:
from langchain import hub
from langchain.prompts import PromptTemplate

prompt_title = "map-prompt"

# 요약문을 작성하기 위한 프롬프트 정의 (직접 프롬프트를 작성하는 경우)
prompt_template = """You are a helpful expert journalist in extracting the main themes from a GIVEN DOCUMENTS below.
Please provide a comprehensive summary of the GIVEN DOCUMENTS in numbered list format.
The summary should cover all the key points and main ideas presented in the original text, while also condensing the information into a concise and easy-to-understand format.
Please ensure that the summary includes relevant details and examples that support the main ideas, while avoiding any unnecessary information or repetition.
The length of the summary should be appropriate for the length and complexity of the original text, providing a clear and accurate overview without omitting any important information.

GIVEN DOCUMENTS:
{docs}

FORMAT:
1. main theme 1
2. main theme 2
3. main theme 3
...

CAUTION:
- DO NOT list more than 5 main themes.

Helpful Answer:
"""
prompt = PromptTemplate.from_template(prompt_template)
prompt


PromptTemplate(input_variables=['docs'], input_types={}, partial_variables={}, template='You are a helpful expert journalist in extracting the main themes from a GIVEN DOCUMENTS below.\nPlease provide a comprehensive summary of the GIVEN DOCUMENTS in numbered list format. \nThe summary should cover all the key points and main ideas presented in the original text, while also condensing the information into a concise and easy-to-understand format. \nPlease ensure that the summary includes relevant details and examples that support the main ideas, while avoiding any unnecessary information or repetition. \nThe length of the summary should be appropriate for the length and complexity of the original text, providing a clear and accurate overview without omitting any important information.\n\nGIVEN DOCUMENTS:\n{docs}\n\nFORMAT:\n1. main theme 1\n2. main theme 2\n3. main theme 3\n...\n\nCAUTION:\n- DO NOT list more than 5 main themes.\n\nHelpful Answer:\n')

In [136]:
hub.push(f"{prompt_title}", prompt)


'https://smith.langchain.com/prompts/map-prompt/28669aaf?organizationId=9437d3d5-0cd1-400d-8617-92cc576bb86a'

## Reduce Prompt

In [138]:
from langchain import hub
from langchain.prompts import PromptTemplate

prompt_title = "reduce-prompt"

# 요약문을 작성하기 위한 프롬프트 정의 (직접 프롬프트를 작성하는 경우)
prompt_template = """You are a helpful expert in summary writing.
You are given numbered lists of summaries.
Extract top 10 most important insights from the summaries.
Then, write a summary of the insights in KOREAN.

LIST OF SUMMARIES:
{doc_summaries}

Helpful Answer:
"""
prompt = PromptTemplate.from_template(prompt_template)
prompt


PromptTemplate(input_variables=['doc_summaries'], input_types={}, partial_variables={}, template='You are a helpful expert in summary writing.\nYou are given numbered lists of summaries.\nExtract top 10 most important insights from the summaries.\nThen, write a summary of the insights in KOREAN.\n\nLIST OF SUMMARIES:\n{doc_summaries}\n\nHelpful Answer:\n')

In [140]:
hub.push(f"{prompt_title}", prompt)


'https://smith.langchain.com/prompts/reduce-prompt/5c5d853c?organizationId=9437d3d5-0cd1-400d-8617-92cc576bb86a'

In [141]:
from langchain import hub
from langchain.prompts import PromptTemplate

prompt_title = "chain-of-density-reduce-korean"

# 요약문을 작성하기 위한 프롬프트 정의 (직접 프롬프트를 작성하는 경우)
prompt_template = """You are a helpful expert in summary writing. You are given lists of summaries.
Please sum up previously summarized sentences according to the following REQUEST.
REQUEST:
1. Summarize the main points in bullet points in KOREAN.
2. Each summarized sentence must start with an emoji that fits the meaning of the each sentence.
3. Use various emojis to make the summary more interesting.
4. MOST IMPORTANT points should be organized at the top of the list.
5. DO NOT include any unnecessary information.

LIST OF SUMMARIES:
{doc_summaries}

Helpful Answer: """
prompt = PromptTemplate.from_template(prompt_template)
prompt


PromptTemplate(input_variables=['doc_summaries'], input_types={}, partial_variables={}, template='You are a helpful expert in summary writing. You are given lists of summaries.\nPlease sum up previously summarized sentences according to the following REQUEST.\nREQUEST:\n1. Summarize the main points in bullet points in KOREAN.\n2. Each summarized sentence must start with an emoji that fits the meaning of the each sentence.\n3. Use various emojis to make the summary more interesting.\n4. MOST IMPORTANT points should be organized at the top of the list.\n5. DO NOT include any unnecessary information.\n\nLIST OF SUMMARIES:\n{doc_summaries}\n\nHelpful Answer: ')

In [142]:
hub.push(f"{prompt_title}", prompt)

'https://smith.langchain.com/prompts/chain-of-density-reduce-korean/45d0ab93?organizationId=9437d3d5-0cd1-400d-8617-92cc576bb86a'

## Metadata Tagger

In [143]:
from langchain import hub
from langchain.prompts import ChatPromptTemplate

prompt_title = "metadata-tagger"

# 요약문을 작성하기 위한 프롬프트 정의 (직접 프롬프트를 작성하는 경우)
prompt_template = """Given the following product review, conduct a comprehensive analysis to extract key aspects mentioned by the customer, with a focus on evaluating the product's design and distinguishing between positive aspects and areas for improvement.
Identify primary features or attributes of the product that the customer appreciated or highlighted, specifically looking for mentions related to the feel of the keys, sound produced by the keys, overall user experience, charging aspect, and the design of the product, etc.
Assess the overall tone of the review (positive, neutral, or negative) based on the sentiment expressed about these attributes.
Additionally, provide a detailed evaluation of the design, outline the positive aspects that the customer enjoyed, and note any areas of improvement or disappointment mentioned.
Extract the customer's rating of the product on a scale of 1 to 5, as indicated at the beginning of the review.
Summarize your findings in a structured JSON format, including an array of keywords, evaluations for design, satisfaction points, improvement areas, the assessed tone, and the numerical rating.

INPUT:
{input}

"""
prompt = ChatPromptTemplate.from_template(prompt_template)
prompt


ChatPromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, template="Given the following product review, conduct a comprehensive analysis to extract key aspects mentioned by the customer, with a focus on evaluating the product's design and distinguishing between positive aspects and areas for improvement. \nIdentify primary features or attributes of the product that the customer appreciated or highlighted, specifically looking for mentions related to the feel of the keys, sound produced by the keys, overall user experience, charging aspect, and the design of the product, etc. \nAssess the overall tone of the review (positive, neutral, or negative) based on the sentiment expressed about these attributes. \nAdditionally, provide a detailed evaluation of the design, outline the positive aspects that the customer enjoyed, and note any areas of

In [145]:
hub.push(f"{prompt_title}", prompt)


'https://smith.langchain.com/prompts/metadata-tagger/d5031809?organizationId=9437d3d5-0cd1-400d-8617-92cc576bb86a'

## Chain of Density 요약

In [146]:
from langchain import hub
from langchain.prompts import ChatPromptTemplate

prompt_title = "chain-of-density-korean"

# 요약문을 작성하기 위한 프롬프트 정의 (직접 프롬프트를 작성하는 경우)
prompt = ChatPromptTemplate.from_template(
    """Article: {ARTICLE}
You will generate increasingly concise, entity-dense summaries of the above article.

Repeat the following 2 steps 5 times.

Step 1. Identify 1-3 informative entities (";" delimited) from the article which are missing from the previously generated summary.
Step 2. Write a new, denser summary of identical length which covers every entity and detail from the previous summary plus the missing entities.

A missing entity is:
- relevant to the main story,
- specific yet concise (100 words or fewer),
- novel (not in the previous summary),
- faithful (present in the article),
- anywhere (can be located anywhere in the article).

Guidelines:

- The first summary should be long (8-10 sentences, ~200 words) yet highly non-specific, containing little information beyond the entities marked as missing. Use overly verbose language and fillers (e.g., "this article discusses") to reach ~200 words.
- Make every word count: rewrite the previous summary to improve flow and make space for additional entities.
- Make space with fusion, compression, and removal of uninformative phrases like "the article discusses".
- The summaries should become highly dense and concise yet self-contained, i.e., easily understood without the article.
- Missing entities can appear anywhere in the new summary.
- Never drop entities from the previous summary. If space cannot be made, add fewer new entities.

Remember, use the exact same number of words for each summary.
Answer in JSON. The JSON should be a list (length 5) of dictionaries whose keys are "Missing_Entities" and "Denser_Summary".
Use only KOREAN language to reply."""
)


In [147]:
hub.push(f"{prompt_title}", prompt)

'https://smith.langchain.com/prompts/chain-of-density-korean/2d444953?organizationId=9437d3d5-0cd1-400d-8617-92cc576bb86a'

## Chain of Density (Korean) - 2

In [148]:
from langchain import hub
from langchain.prompts import ChatPromptTemplate

prompt_title = "chain-of-density-map-korean"

# 요약문을 작성하기 위한 프롬프트 정의 (직접 프롬프트를 작성하는 경우)
prompt = ChatPromptTemplate.from_template(
    """Article: {ARTICLE}
You will generate increasingly concise, entity-dense summaries of the above article.

Repeat the following 2 steps 3 times.

Step 1. Identify 1-3 informative entities (";" delimited) from the article which are missing from the previously generated summary.
Step 2. Write a new, denser summary of identical length which covers every entity and detail from the previous summary plus the missing entities.

A missing entity is:
- relevant to the main story,
- specific yet concise (100 words or fewer),
- novel (not in the previous summary),
- faithful (present in the article),
- anywhere (can be located anywhere in the article).

Guidelines:

- The first summary should be long (8-10 sentences, ~200 words) yet highly non-specific, containing little information beyond the entities marked as missing. Use overly verbose language and fillers (e.g., "this article discusses") to reach ~200 words.
- Make every word count: rewrite the previous summary to improve flow and make space for additional entities.
- Make space with fusion, compression, and removal of uninformative phrases like "the article discusses".
- The summaries should become highly dense and concise yet self-contained, i.e., easily understood without the article.
- Missing entities can appear anywhere in the new summary.
- Never drop entities from the previous summary. If space cannot be made, add fewer new entities.

Remember, use the exact same number of words for each summary.
Answer "Missing Entities" and "Denser_Summary" as in TEXT format.
Use only KOREAN language to reply."""
)


In [149]:
hub.push(f"{prompt_title}", prompt)

'https://smith.langchain.com/prompts/chain-of-density-map-korean/0b372530?organizationId=9437d3d5-0cd1-400d-8617-92cc576bb86a'

## RAG 문서 프롬프트

In [150]:
prompt_title = "rag-prompt-korean"


In [151]:
system = """당신은 질문-답변(Question-Answering)을 수행하는 친절한 AI 어시스턴트입니다. 당신의 임무는 주어진 문맥(context) 에서 주어진 질문(question) 에 답하는 것입니다.
검색된 다음 문맥(context) 을 사용하여 질문(question) 에 답하세요. 만약, 주어진 문맥(context) 에서 답을 찾을 수 없다면, 답을 모른다면 `주어진 정보에서 질문에 대한 정보를 찾을 수 없습니다` 라고 답하세요.
한글로 답변해 주세요. 단, 기술적인 용어나 이름은 번역하지 않고 그대로 사용해 주세요. Don't narrate the answer, just answer the question. Let's think step-by-step."""

human = """#Question:
{question}

#Context:
{context}

#Answer:"""

from langchain.prompts import ChatPromptTemplate


prompt = ChatPromptTemplate.from_messages([("system", system), ("human", human)])


In [152]:
hub.push(f"{prompt_title}", prompt, parent_commit_hash="latest")

'https://smith.langchain.com/prompts/rag-prompt-korean/cbfb554c?organizationId=9437d3d5-0cd1-400d-8617-92cc576bb86a'